# The Associated Press and Life Expectancy

**Story:** [AP analysis: Unemployment, income affect life expectancy](https://www.apnews.com/66ac44186b6249709501f07a7eab36da)

**Author:** Nicky Forster, Associated Press

**Topics:** Census Data, Linear Regression

**Datasets**

* **R12221544_SL140.csv:** ACS 2015 5-year, tract level, from [Social Explorer](https://www.socialexplorer.com)
    - Table B23025: Employment Status
    - **R12221544.txt** is the data dictionary
* **R12221544_SL140.csv:** ACS 2015 5-year, tract level, from [Social Explorer](https://www.socialexplorer.com)
    - Table B23025: Employment Status
    - Table B06009: Educational Attainment
    - Table B03002: Race
    - Table B19013: Median income
    - Table C17002: Ratio of income to poverty level
    - **R12221544.txt** is the data dictionary
* **US_A.CSV:** life expectancy by census tract, from [USALEEP](https://www.cdc.gov/nchs/nvss/usaleep/usaleep.html)
    - **Record_Layout_CensusTract_Life_Expectancy.pdf** is data dictionary

# What's the story?

We're trying to figure out how the **life expectancy in a census tract** is related to other factors like unemployment, income, and others.

# PREPWORK BONUS!

Download the data yourself from Social Explorer and USALEEP (linked above) instead of relying on the data included.

## Reading in our data

### Read in `USA_A.CSV`

Rename any columns with weird or not-understandable names as something more descriptive.

In [11]:
import pandas as pd
import requests
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time
import pandas as pd
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
sns.set(style='whitegrid',palette='husl')
matplotlib.rcParams['pdf.fonttype'] = 42

In [2]:
df=pd.read_csv('US_A.CSV',encoding='latin-1')

In [3]:
df=df.rename(columns={'e(0)':'life_expectancy','se(e(0))':'std_error'})

In [4]:
df.head(3)

,Tract ID,STATE2KX,CNTY2KX,TRACT2KX,life_expectancy,std_error,Abridged life table flag
0,1001020100,1,1,20100,73.1,2.2348,3
1,1001020200,1,1,20200,76.9,3.3453,3
2,1001020400,1,1,20400,75.4,1.0216,3


### Open `R12221544_SL140.csv`

You'll need to give an option to `pd.read_csv` to make sure it's read in successfully.

In [5]:
df1=pd.read_csv('R12221544_SL140.csv',encoding='latin-1')

#### Filter out any columns we aren't interestd in

In [6]:
df1 = df1[['Geo_FIPS', 'Geo_GEOID', 'Geo_NAME', 'Geo_STUSAB', 'Geo_GEOCOMP', 'Geo_FILEID', 'Geo_LOGRECNO', 'Geo_US',
      'Geo_REGION', 'Geo_DIVISION', 'Geo_STATECE', 'Geo_STATE', 'Geo_COUNTY','Geo_TRACT', 'ACS15_5yr_B23025001',
      'ACS15_5yr_B23025002', 'ACS15_5yr_B23025003', 'ACS15_5yr_B23025004',
      'ACS15_5yr_B23025005', 'ACS15_5yr_B23025006', 'ACS15_5yr_B23025007',
      'ACS15_5yr_B23025001s', 'ACS15_5yr_B23025002s', 'ACS15_5yr_B23025003s',
      'ACS15_5yr_B23025004s', 'ACS15_5yr_B23025005s', 'ACS15_5yr_B23025006s',
      'ACS15_5yr_B23025007s']]
df1 = df1.rename(columns={'Geo_TRACT':'TRACT', 'ACS15_5yr_B23025001':'Pop', 'ACS15_5yr_B23025002':'Pop_in_Labor_Force', 'ACS15_5yr_B23025003':'Pop_in_Civilian_Labor_Force', 'ACS15_5yr_B23025004':'Pop_in_Civilian_Labor_Force_employed',
      'ACS15_5yr_B23025005':'Pop_in_Civilian_Labor_Force_unemployed', 'ACS15_5yr_B23025006':'Pop_in_Labor_Force_Armed_Forces', 'ACS15_5yr_B23025007':'Pop_Not_in_Labor_Force',
      'ACS15_5yr_B23025001s':'Error_Pop', 'ACS15_5yr_B23025002s':'Error_Pop_in_Labor_Force', 'ACS15_5yr_B23025003s':'Error_Pop_in_Civilian_Labor_Force',
      'ACS15_5yr_B23025004s':'Error_Pop_in_Civilian_Labor_Force_Employed', 'ACS15_5yr_B23025005s':'Error_Pop_in_Civilian_Labor_Force_Unemployed', 'ACS15_5yr_B23025006s':'Error_Pop_in_Labor_Force_Armed_Forces',
      'ACS15_5yr_B23025007s':'Error_Pop_Not_in_Labor_Force'})
df1

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_STUSAB,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,Geo_REGION,Geo_DIVISION,...,Pop_in_Civilian_Labor_Force_unemployed,Pop_in_Labor_Force_Armed_Forces,Pop_Not_in_Labor_Force,Error_Pop,Error_Pop_in_Labor_Force,Error_Pop_in_Civilian_Labor_Force,Error_Pop_in_Civilian_Labor_Force_Employed,Error_Pop_in_Civilian_Labor_Force_Unemployed,Error_Pop_in_Labor_Force_Armed_Forces,Error_Pop_Not_in_Labor_Force
0,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",al,0,ACSSF,1760,NaN,NaN,NaN,...,54,0,557,92.121212,85.454545,85.454545,83.636364,18.787879,6.666667,67.878788
1,1001020200,14000US01001020200,"Census Tract 202, Autauga County, Alabama",al,0,ACSSF,1761,NaN,NaN,NaN,...,116,15,847,143.030303,115.151515,114.545455,107.272727,38.181818,14.545455,86.666667
2,1001020300,14000US01001020300,"Census Tract 203, Autauga County, Alabama",al,0,ACSSF,1762,NaN,NaN,NaN,...,91,8,990,169.090909,132.121212,134.545455,123.030303,31.515152,8.484848,120.606061
3,1001020400,14000US01001020400,"Census Tract 204, Autauga County, Alabama",al,0,ACSSF,1763,NaN,NaN,NaN,...,216,15,1411,197.575758,157.575758,161.818182,132.121212,58.787879,14.545455,127.878788
4,1001020500,14000US01001020500,"Census Tract 205, Autauga County, Alabama",al,0,ACSSF,1764,NaN,NaN,NaN,...,221,203,2737,321.818182,339.393939,356.969697,369.090909,89.090909,103.030303,273.939394
5,1001020600,14000US01001020600,"Census Tract 206, Autauga County, Alabama",al,0,ACSSF,1765,NaN,NaN,NaN,...,190,52,1053,206.060606,160.606061,158.787879,139.393939,63.030303,30.303030,151.515152
6,1001020700,14000US01001020700,"Census Tract 207, Autauga County, Alabama",al,0,ACSSF,1766,NaN,NaN,NaN,...,150,0,796,163.636364,130.303030,130.303030,125.454545,47.878788,6.666667,80.606061
7,1001020801,14000US01001020801,"Census Tract 208.01, Autauga County, Alabama",al,0,ACSSF,1767,NaN,NaN,NaN,...,135,28,854,127.272727,124.242424,122.424242,136.969697,47.878788,13.939394,117.575758
8,1001020802,14000US01001020802,"Census Tract 208.02, Autauga County, Alabama",al,0,ACSSF,1768,NaN,NaN,NaN,...,415,124,3201,379.393939,269.696970,276.363636,250.303030,112.121212,47.878788,265.454545
9,1001020900,14000US01001020900,"Census Tract 209, Autauga County, Alabama",al,0,ACSSF,1769,NaN,NaN,NaN,...,194,0,1799,303.636364,218.787879,218.787879,214.545455,91.515152,9.696970,181.818182


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74001 entries, 0 to 74000
Data columns (total 28 columns):
Geo_FIPS                                        74001 non-null int64
Geo_GEOID                                       74001 non-null object
Geo_NAME                                        74001 non-null object
Geo_STUSAB                                      74001 non-null object
Geo_GEOCOMP                                     74001 non-null int64
Geo_FILEID                                      74001 non-null object
Geo_LOGRECNO                                    74001 non-null int64
Geo_US                                          0 non-null float64
Geo_REGION                                      0 non-null float64
Geo_DIVISION                                    0 non-null float64
Geo_STATECE                                     0 non-null float64
Geo_STATE                                       74001 non-null int64
Geo_COUNTY                                      74001 non-null int6

#### Create a new column for percent unemployment

We'll be using the total population in the census tract as the baseline for employment.

In [7]:
df1['pct_unemployed']=df1['Pop_in_Civilian_Labor_Force_unemployed']/df1['Pop']

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_STUSAB,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,Geo_REGION,Geo_DIVISION,...,Pop_in_Labor_Force_Armed_Forces,Pop_Not_in_Labor_Force,Error_Pop,Error_Pop_in_Labor_Force,Error_Pop_in_Civilian_Labor_Force,Error_Pop_in_Civilian_Labor_Force_Employed,Error_Pop_in_Civilian_Labor_Force_Unemployed,Error_Pop_in_Labor_Force_Armed_Forces,Error_Pop_Not_in_Labor_Force,pct_unemployed
0,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",al,0,ACSSF,1760,NaN,NaN,NaN,...,0,557,92.121212,85.454545,85.454545,83.636364,18.787879,6.666667,67.878788,0.034749
168,1033020100,14000US01033020100,"Census Tract 201, Colbert County, Alabama",al,0,ACSSF,1928,NaN,NaN,NaN,...,0,1505,135.151515,115.151515,115.151515,108.484848,53.939394,6.666667,124.848485,0.046770
229,1045020100,14000US01045020100,"Census Tract 201, Dale County, Alabama",al,0,ACSSF,1989,NaN,NaN,NaN,...,0,1359,156.363636,143.030303,143.030303,132.727273,40.606061,6.666667,115.757576,0.040129
326,1057020100,14000US01057020100,"Census Tract 201, Fayette County, Alabama",al,0,ACSSF,2086,NaN,NaN,NaN,...,0,1228,179.393939,139.393939,139.393939,143.030303,30.303030,6.666667,118.787879,0.036140
1147,1127020100,14000US01127020100,"Census Tract 201, Walker County, Alabama",al,0,ACSSF,2907,NaN,NaN,NaN,...,0,1271,179.393939,122.424242,122.424242,128.484848,69.090909,6.666667,130.303030,0.077985
1452,4012020100,14000US04012020100,"Census Tract 201, La Paz County, Arizona",az,0,ACSSF,823,NaN,NaN,NaN,...,0,1305,175.151515,106.666667,106.666667,98.787879,34.545455,7.272727,130.303030,0.034038
3018,5033020100,14000US05033020100,"Census Tract 201, Crawford County, Arkansas",ar,0,ACSSF,3525,NaN,NaN,NaN,...,7,2470,264.242424,232.121212,231.515152,219.393939,54.545455,4.848485,238.787879,0.027435
3136,5059020100,14000US05059020100,"Census Tract 201, Hot Spring County, Arkansas",ar,0,ACSSF,3643,NaN,NaN,NaN,...,0,2289,216.363636,214.545455,214.545455,201.212121,57.575758,9.696970,167.878788,0.030437
3243,5091020100,14000US05091020100,"Census Tract 201, Miller County, Arkansas",ar,0,ACSSF,3750,NaN,NaN,NaN,...,0,957,176.363636,137.575758,137.575758,127.878788,65.454545,6.666667,121.212121,0.089664
9916,6075020100,14000US06075020100,"Census Tract 201, San Francisco County, Califo...",ca,0,ACSSF,9059,NaN,NaN,NaN,...,0,1797,323.030303,257.575758,257.575758,229.090909,121.212121,10.303030,218.787879,0.073310


In [8]:
df.head(3)

,Tract ID,STATE2KX,CNTY2KX,TRACT2KX,life_expectancy,std_error,Abridged life table flag
0,1001020100,1,1,20100,73.1,2.2348,3
1,1001020200,1,1,20200,76.9,3.3453,3
2,1001020400,1,1,20400,75.4,1.0216,3


## Merging the data

Merge the dataframes together based on their census tract.

In [9]:
merged=df.merge(df1,left_on='TRACT2KX',right_on='TRACT')

In [18]:
merged.head(3)

,Tract ID,STATE2KX,CNTY2KX,TRACT2KX,life_expectancy,std_error,Abridged life table flag,Geo_FIPS,Geo_GEOID,Geo_NAME,...,Pop_in_Labor_Force_Armed_Forces,Pop_Not_in_Labor_Force,Error_Pop,Error_Pop_in_Labor_Force,Error_Pop_in_Civilian_Labor_Force,Error_Pop_in_Civilian_Labor_Force_Employed,Error_Pop_in_Civilian_Labor_Force_Unemployed,Error_Pop_in_Labor_Force_Armed_Forces,Error_Pop_Not_in_Labor_Force,pct_unemployed
0,1001020100,1,1,20100,73.1,2.2348,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,0,557,92.121212,85.454545,85.454545,83.636364,18.787879,6.666667,67.878788,0.034749
1,1001020100,1,1,20100,73.1,2.2348,3,1033020100,14000US01033020100,"Census Tract 201, Colbert County, Alabama",...,0,1505,135.151515,115.151515,115.151515,108.484848,53.939394,6.666667,124.848485,0.046770
2,1001020100,1,1,20100,73.1,2.2348,3,1045020100,14000US01045020100,"Census Tract 201, Dale County, Alabama",...,0,1359,156.363636,143.030303,143.030303,132.727273,40.606061,6.666667,115.757576,0.040129


In [10]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2729847 entries, 0 to 2729846
Data columns (total 36 columns):
Tract ID                                        int64
STATE2KX                                        int64
CNTY2KX                                         int64
TRACT2KX                                        int64
life_expectancy                                 float64
std_error                                       float64
Abridged life table flag                        int64
Geo_FIPS                                        int64
Geo_GEOID                                       object
Geo_NAME                                        object
Geo_STUSAB                                      object
Geo_GEOCOMP                                     int64
Geo_FILEID                                      object
Geo_LOGRECNO                                    int64
Geo_US                                          float64
Geo_REGION                                      float64
Geo_DIVISIO

## Running the regression

Using the `statsmodels` package, run a linear regression to find the coefficient relating unemployment and life expectancy.

In [16]:
X = merged[['pct_unemployed']]*100
X = sm.add_constant(X)
mod = sm.OLS(merged.life_expectancy, X, missing='drop')
res = mod.fit()

In [17]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        life_expectancy   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2293.
Date:                Wed, 17 Jul 2019   Prob (F-statistic):               0.00
Time:                        16:22:43   Log-Likelihood:            -7.7548e+06
No. Observations:             2728570   AIC:                         1.551e+07
Df Residuals:                 2728568   BIC:                         1.551e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             77.2391      0.005   1.59e+04      0.000      77.230      77.249
pct_unemployed    -0.0365      0.001    -47.884      0.000      -0.038      -0.035
==============================================================================
Omnibus:                     8361.520   Durbin-Watson:                   0.038
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9710.959
Skew:                          -0.082   Prob(JB):                         0.00
Kurtosis:                       3.242   Cond. No.                         12.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Translate that into the form **"every X percentage point change in unemployment translates to a Y change in life expectancy"**

In [ ]:
#every 25 percentage point change in unemployment translates to a 1 year decrease in life expectancy

## Bringing more columns into the mix

Only dealing with unemployment seems kind of narrow-minded, let's expand our reach a bit.

### Read in `R12221550_SL140.csv`

It's also from the Census, and has many, many, many more columns available to you compared to the list dataset.

In [20]:
df2=pd.read_csv('R12221550_SL140.csv',encoding='latin-1')

Using this census data, create a new dataframe that includes the following columns:

* Percent unemployed
* Percents Black, White, and Hispanic
* Median Income (in increments of 10,000 dollars)
* Percent of the population with less than a high school education
* Percent of the population between 1-1.5x the poverty line

If you have to many any editorial decisions about which columns you choose or how you do your math, please explain them.

In [32]:
df2 = df2[['Geo_FIPS', 'Geo_GEOID', 'Geo_NAME', 'Geo_STUSAB', 'Geo_GEOCOMP', 'Geo_FILEID', 'Geo_LOGRECNO', 'Geo_US',
      'Geo_REGION', 'Geo_DIVISION', 'Geo_STATECE', 'Geo_STATE', 'Geo_COUNTY','Geo_TRACT', 'ACS15_5yr_B03002003',
      'ACS15_5yr_B03002004', 'ACS15_5yr_B03002012', 'ACS15_5yr_B23025004',
      'ACS15_5yr_B19013001', 'ACS15_5yr_B06009008', 'ACS15_5yr_C17002004',
      'ACS15_5yr_C17002005']]
df2 = df2.rename(columns={'Geo_TRACT':'TRACT', 'ACS15_5yr_B03002004':'black_pop', 'ACS15_5yr_B03002012':'hispanic_pop', 
      'ACS15_5yr_B19013001':'median_income', 'ACS15_5yr_B06009008':'Pop_less_hs', 'ACS15_5yr_C17002004':'pop_poverty1124',
      'ACS15_5yr_C17002005':'pop_poverty1245', 'ACS15_5yr_B03002003':'white_pop'})


In [36]:
df2['median_income']=df2['median_income']/10000

### Join your datasets

Combine your life expectancy dataset with this census dataset to create a new dataframe.

In [37]:
merged1=pd.merge(merged,df2, on='Geo_FIPS')

In [38]:
merged1

,Tract ID,STATE2KX,CNTY2KX,TRACT2KX,life_expectancy,std_error,Abridged life table flag,Geo_FIPS,Geo_GEOID_x,Geo_NAME_x,...,Geo_COUNTY_y,TRACT_y,white_pop,black_pop,hispanic_pop,ACS15_5yr_B23025004,median_income,Pop_less_hs,pop_poverty1124,pop_poverty1245
0,1001020100,1,1,20100,73.1,2.2348,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,1,20100,1703,150,17,943,6.1838,118.0,81,101
1,1033020100,1,33,20100,72.3,1.4305,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,1,20100,1703,150,17,943,6.1838,118.0,81,101
2,1045020100,1,45,20100,75.0,1.5468,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,1,20100,1703,150,17,943,6.1838,118.0,81,101
3,1057020100,1,57,20100,72.6,1.4231,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,1,20100,1703,150,17,943,6.1838,118.0,81,101
4,1127020100,1,127,20100,68.6,1.3919,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,1,20100,1703,150,17,943,6.1838,118.0,81,101
5,4012020100,4,12,20100,78.9,1.3106,2,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,1,20100,1703,150,17,943,6.1838,118.0,81,101
6,5033020100,5,33,20100,76.4,1.8804,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,1,20100,1703,150,17,943,6.1838,118.0,81,101
7,5059020100,5,59,20100,73.4,1.2765,1,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,1,20100,1703,150,17,943,6.1838,118.0,81,101
8,5091020100,5,91,20100,72.9,1.3771,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,1,20100,1703,150,17,943,6.1838,118.0,81,101
9,6075020100,6,75,20100,75.7,2.0606,3,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama",...,1,20100,1703,150,17,943,6.1838,118.0,81,101


In [48]:
merged1['pct_black']=merged1['black_pop']/merged1['Pop']*100
merged1['pct_hispanic']=merged1['hispanic_pop']/merged1['Pop']*100
merged1['pct_white']=merged1['hispanic_pop']/merged1['Pop']*100
merged1['pct_unemployed']=merged1['pct_unemployed']*100

In [45]:
merged1.dtypes

Tract ID                                          int64
STATE2KX                                          int64
CNTY2KX                                           int64
TRACT2KX                                          int64
life_expectancy                                 float64
std_error                                       float64
Abridged life table flag                          int64
Geo_FIPS                                          int64
Geo_GEOID_x                                      object
Geo_NAME_x                                       object
Geo_STUSAB_x                                     object
Geo_GEOCOMP_x                                     int64
Geo_FILEID_x                                     object
Geo_LOGRECNO_x                                    int64
Geo_US_x                                        float64
Geo_REGION_x                                    float64
Geo_DIVISION_x                                  float64
Geo_STATECE_x                                   

In [49]:
merged1['pct_pov']=(merged1['pop_poverty1124']+merged1['pop_poverty1245'])/merged['Pop']*100

## Running your multivariate regression

Using the `statsmodels` package and this new dataframe, run a multivariate linear regression to find the coefficient relating your columns and life expectancy.

In [51]:
X = sm.add_constant(merged1[['pct_unemployed', 'white_pop','black_pop','hispanic_pop','Pop_less_hs','pct_pov', 'median_income']])
y = merged1['life_expectancy']
mod = sm.OLS(y, X, missing='drop')
res = mod.fit()
res.summary()

MissingDataError: exog contains inf or nans

Translate some of your coefficients into the form **"every X percentage point change in unemployment translates to a Y change in life expectancy."** Do this with numbers that are meaningful, and in a way that is easily understandable to your reader.